In [17]:
# importing libraries
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add,Lambda
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import tensorflow as tf
import cv2
from keras.layers import add
from keras.applications import VGG19
import keras.backend as K
from tqdm import tqdm
from numpy.random import randint

In [18]:

def res_block_gen(gen, kernal_size, filters, strides):
#     gen = model
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(gen)
    model = BatchNormalization(momentum = 0.5)(model)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)

    model = Add()([gen, model])

    return model

# def SubpixelConv2D(input_shape, scale=2):
#     """
#     Keras layer to do subpixel convolution.
#     NOTE: Tensorflow backend only. Uses tf.depth_to_space
#     Ref:
#         [1] Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network
#             Shi et Al.
#             https://arxiv.org/abs/1609.05158
#     :param input_shape: tensor shape, (batch, height, width, channel)
#     :param scale: upsampling scale. Default=4
#     :return:
#     """
#     # upsample using depth_to_space
#     def subpixel_shape(input_shape):
#         dims = [input_shape[0],
#                 input_shape[1] * scale,
#                 input_shape[2] * scale,
#                 int(input_shape[3] / (scale ** 2))]
#         output_shape = tuple(dims)
#         return output_shape

#     def subpixel(x):
#         return tf.depth_to_space(x, scale)


#     return Lambda(subpixel, output_shape=subpixel_shape, name='subpixel')

# def content_loss(hr, sr):
#     #vgg = srgan.vgg_54()
#     vgg = VGG(image_HR_shape).vgg_19()
#     sr = preprocess_input(sr)
#     hr = preprocess_input(hr)
#     sr_features = vgg(sr)
#     hr_features = vgg(hr)
#     return mean_squared_error(hr_features, sr_features)

def SubpixelConv2D(scale):
        return Lambda(lambda x: tf.depth_to_space(x, scale))


def plotLoss(epoch,dLosses,gLosses):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('/home/vidyach/images/pixel/plots/dcgan_%d_loss_epoch.png' % epoch)
    
def up_sampling_block(model, kernal_size, filters, strides):
   # model = UpSampling2D(size = 2)(model)
    #model = Conv2DTranspose(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    
    model = SubpixelConv2D(2)(model)
    
    model = LeakyReLU(alpha = 0.2)(model)

    return model

def discriminator_block(model, filters, kernel_size, strides):

    model = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = LeakyReLU(alpha = 0.2)(model)

    return model


def get_gan_network(discriminator, shape, generator, optimizer, vgg):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    x_feat = vgg(x)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x_feat,gan_output])
    gan.compile(loss=["mse", "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)

    return gan

def normalize(input_data):

    return (input_data.astype(np.float32) - 127.5)/127.5 

def datagen(batchSize,filesList,filePath):
    while(True):
        files = np.random.choice(filesList,batchSize,replace=False)
#         print(files)
#         print("----------")
        X_train_HR = []
        X_train_LR = []
        for file in files:
            image = cv2.imread(filePath + "/" + file)
#             print(filePath + "/" + file)
#             print(image)
            image_HR = cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
            image_HR = normalize(image_HR)
            #image_HR = image_HR / 255.0
            X_train_HR.append(image_HR)
            
            image_LR = cv2.resize(image,(56,56),interpolation = cv2.INTER_CUBIC)
            #image_LR = image_LR / 255.0
            image_LR = normalize(image_LR)
            
            X_train_LR.append(image_LR)
            
        X_train_HR = np.array(X_train_HR)
        X_train_LR = np.array(X_train_LR)
        yield X_train_LR,X_train_HR
        
      
    

    
def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8)


def plotGeneratedImages(output_dir,epoch,fake_img,low_img,hr_img,dim=(1, 3),figsize=(15, 5)):
    
    examples = hr_img.shape[0]
    print(examples)
    value = randint(0, examples)
    image_batch_hr = denormalize(hr_img)
    image_batch_lr = denormalize(low_img)
    #gen_img = generator.predict(image_batch_lr)
    #generated_image = denormalize(gen_img)
    generated_image = denormalize(fake_img)
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[value], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[value], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[value], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir + 'generated_image_%d.png' % epoch)
    
    
def saveModels(epoch,generator,discriminator):
    fol = '/home/vidyach/models/pixel/'
#     if not os.path.exists(fol):
#         os.makedirs(fol)
    generator.save(fol+'dcgan_generator_epoch_%d.h5' % epoch)
    discriminator.save(fol+'dcgan_discriminator_epoch_%d.h5' % epoch)


In [19]:
class Generator():
    def __init__(self, noise_shape):
        self.noise_shape = noise_shape
        
    def gen(self):
        gen_Input = Input(shape = self.noise_shape)
        
        
        model = Conv2D(filters = 64,kernel_size = 9 ,strides=1,padding = "same") (gen_Input)
        model = temp = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
        
        for i in range(16):
            model = res_block_gen(model,3,64,1)
          
        model = Conv2D(filters = 64,kernel_size=3,strides=1,padding="same")(model)
        model = BatchNormalization()(model)
        model = Add()([temp,model])
            
        for i in range(2):
            model = up_sampling_block(model,3,256,1)
            
        
        
        model = Conv2D(filters = 3,kernel_size=9,strides=1,padding="same")(model)
        model = Activation('tanh')(model)
        
        generator_model = Model(inputs = gen_Input, outputs = model)
        
        return generator_model
    

class Disciminator():
    def __init__(self, image_shape):
        
        self.image_shape = image_shape
        
        
    def disc(self):
        disc_Input = Input(shape = self.image_shape)
        
        model = Conv2D(filters = 64,kernel_size = 3 ,strides=1,padding = "same") (disc_Input)
        model = LeakyReLU(alpha = 0.2)(model)
        
        
        model = discriminator_block(model, 64, 3, 2)
        model = discriminator_block(model, 128, 3, 1)
        model = discriminator_block(model, 128, 3, 2)
        model = discriminator_block(model, 256, 3, 1)
        model = discriminator_block(model, 256, 3, 2)
        model = discriminator_block(model, 512, 3, 1)
        model = discriminator_block(model, 512, 3, 2)
        
        model = Flatten()(model)
        
        model = Dense(1024)(model)
        model = LeakyReLU(alpha = 0.2)(model)
        
        model = Dense(1)(model)
        model = Activation('sigmoid')(model)
        
        disciminator_model = Model(inputs = disc_Input,output = model)
        
        
        return disciminator_model
    

        
class VGG(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape

    # computes VGG loss or content loss
    def vgg_19(self):
        
        img = Input(shape = self.image_shape)
        vgg = VGG19(weights="imagenet")
        vgg.outputs = [vgg.layers[9].output]
        model = Model(inputs = img, outputs=vgg(img))
        model.trainable = False
        return model

In [20]:
# extracting the dataset
# filePath_train_LR = 'Desktop/spring2019/Aml/proj3/dataset/DIV2K_train_LR_x8'
# filePath_val_LR = 'Desktop/spring2019/Aml/proj3/dataset/DIV2K_valid_LR_x8'
filePath_train_HR = '/home/vidyach/DIV2K_train_HR'
#filePath_val_HR = 'Desktop/spring2019/Aml/proj3/dataset/DIV2K_valid_HR'
#filePath_train_HR = 'Desktop/spring2019/Aml/proj3/dataset/DIV2K_train_HR'

train_HR = [f for f in os.listdir(filePath_train_HR) ]
#valid_HR = [f for f in os.listdir(filePath_val_HR) ]



In [25]:
# datagenObj = datagen(10,train_HR,filePath_train_HR)
# temp = next(datagenObj)
# print(temp[0][0])

In [26]:
#print(temp[0][1])

In [ ]:
image_HR_shape = (224,224,3)
image_LR_shape = (56,56,3)
optimizer = Adam(0.0002, 0.5)
# optimizer = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
no_of_epoch = 2000
batch_count = 10
d_loss = []
g_loss = []
filePathImage = "/home/vidyach/images/pixel/plots/"



vgg = VGG(image_HR_shape).vgg_19()
vgg.compile(loss = "mse", optimizer = optimizer)

generator = Generator(image_LR_shape).gen()

discriminator = Disciminator(image_HR_shape).disc()
discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)

gan = get_gan_network(discriminator, image_LR_shape, generator, optimizer, vgg)

datagenObj = datagen(batch_count,train_HR,filePath_train_HR)

for e in range(1,no_of_epoch):
    for _ in tqdm(range(batch_count)):
        lr_img,hr_img = next(datagenObj)
        fake_img = generator.predict(lr_img)

        real_data_Y = np.ones(batch_count) - np.random.random_sample(batch_count)*0.2
        fake_data_Y = np.random.random_sample(batch_count)*0.2
        dloss = 0
        gan_loss = 0

        if(e%2==0):
        # training the disciminator
            discriminator.trainable = True

            d_loss_real = discriminator.train_on_batch(hr_img, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(fake_img, fake_data_Y)
            dloss = 0.5 * np.add(d_loss_fake, d_loss_real)
        else:
            # training the combined network
            hr_feat = vgg.predict(hr_img)
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(lr_img, [hr_feat,real_data_Y])

    d_loss.append(dloss)
    g_loss.append(gan_loss)


    if e == 1 or (e > 100 and e % 200 == 0):
        saveModels(e,generator,discriminator)
        plotGeneratedImages(filePathImage,e,fake_img,lr_img,hr_img)

#print("--------------training complete-----------------")

In [ ]:
# train()
plotLoss(e,d_loss,g_loss)

In [ ]:
# validation the trained model

In [ ]:
# filePath_valid_HR = '/home/vidyach/DIV2K_valid_HR'
# filePathImage_valid = "/home/vidyach/images/plots/valid"
# valid_HR = [f for f in os.listdir(filePath_valid_HR)]

# daagenObjValid = datagen(10,valid_HR,filePath_valid_HR)
# epoch_size = 50

# for e in range(1,epoch_size):
#     lr_img,hr_img = next(daagenObjValid)
#     gen_images = generator(lr_img)
    
    
#     if e==1 or e%10==0:
#         plotGeneratedImages(filePathImage_valid,e,gen_images,lr_img,hr_img)
# print("----valid complete----")
    
    
